In [128]:
%run /home/jovyan/data_loading_tools.ipynb
%run /home/jovyan/plotting_tools.ipynb

In [129]:
# Client täytyy käynnistää ekana
client = start_client()

In [130]:
# Alustetaan datan hakua
parameters = ['Temperature']
time_range = 10                   # Määrittää kuinka monelta päivältä dataa haetaan tästä hetkestä eteenpäin
model = 'EC'

delayed_data_ec = get_model_data_delay(parameters, time_range, model)
EC_params = merge_model_delayed(client, delayed_data_ec)

Done!


In [131]:
EC_params.head().compute()

<xarray.Dataset>
Dimensions:            (time: 5, y: 5, x: 5)
Coordinates:
  * time               (time) datetime64[ns] 2024-09-11 ... 2024-09-11T04:00:00
  * y                  (y) float32 -3.883e+06 -3.876e+06 ... -3.854e+06
  * x                  (x) float32 -1.01e+06 -1.003e+06 ... -9.806e+05
    lat                (y, x) float32 52.7 52.72 52.73 ... 52.98 53.0 53.01
    lon                (y, x) float32 5.421 5.523 5.625 ... 5.519 5.621 5.724
Data variables:
    crs                int16 -32767
    air_temperature_4  (time, y, x) float32 284.3 284.7 285.7 ... 283.9 284.3
Attributes:
    Conventions:  CF-1.6
    title:        <title>
    institution:  fmi.fi
    source:       <producer>

In [132]:
T2m = EC_params['air_temperature_4'].compute()

In [133]:
# Satunnaista testailua serveriltä
print(EC_params['air_temperature_4'].max(dim="time").values.compute())
print(EC_params['air_temperature_4'].groupby("time.hour").mean().values.compute())
print(EC_params['air_temperature_4'].resample(time="1D").compute())


[[293.05    292.93    294.13    ... 302.02    302.03    301.63   ]
 [292.85    293.63    294.49    ... 301.83    301.63    301.63   ]
 [293.71    293.85    294.97998 ... 301.72998 301.72    301.72998]
 ...
 [281.08    281.18    281.38    ... 285.91    286.1     285.65   ]
 [280.78    280.97998 281.09    ... 285.81    285.97    286.09   ]
 [280.58    280.68    280.78    ... 285.83    285.71    285.71   ]]
[[[287.7891  287.05637 285.83908 ... 287.38818 287.6082  287.59003]
  [287.52365 286.73727 286.1191  ... 287.41092 287.24    287.13547]
  [287.28543 285.8491  284.86545 ... 287.23633 287.13815 287.03183]
  ...
  [278.37088 278.42728 278.47272 ... 284.66092 284.64908 284.63998]
  [278.25638 278.28635 278.33273 ... 284.65906 284.64453 284.65906]
  [278.13092 278.15817 278.2009  ... 284.66818 284.65    284.6409 ]]

 [[287.83362 287.08694 285.55457 ... 287.4251  287.70667 287.6962 ]
  [287.45984 286.89197 286.1471  ... 287.44952 287.32132 287.15588]
  [287.22775 286.0256  284.78952 ... 287

In [134]:
# Satunnaista testailua lokaalilla

T_max = (T2m.max(dim="time").compute())
#print(T_max.values)
T_hour = (T2m.groupby("time.hour").mean().compute())
#print(T_hour.values)
T_days = (T2m.resample(time="1D"))
#print(T_days)

#print(T2m.isel(time=5).values)


# Meteorologit voi aloittaa tekemisen tästä kohtaa. Ylimpänä oleva "solu" pitää ajaa, jotta saadaan metodit käyttöön

In [135]:
# Lasketaan tähän keskiarvoja.
# Kelluva keskiarvo

meanT = xr.full_like(T2m,0)          # kopio alk.per 2m lämpötila-arraysta, johon voidaan tallentaa aina seuraavan 6h maksimit
for x in range(len(T2m.time)):       # Loopataan jokaisen ajanhetken yli
    end = x+5                        # seuraava 6h
    slice = T2m[x:end,:,:]           # seuraava 6h slice
    tmean = slice.mean(dim='time')   # otetaan siitä maksimi
    meanT[x,:,:] = tmean.data        # tallennetaan maksimit oikeaan paikkaan datassa
print(meanT)

<xarray.DataArray 'air_temperature_4' (time: 259, y: 280, x: 255)>
array([[[285.498  , 285.702  , 285.388  , ..., 286.824  , 288.198  ,
         287.998  ],
        [284.898  , 285.474  , 285.746  , ..., 287.77   , 287.776  ,
         287.452  ],
        [285.126  , 284.902  , 284.612  , ..., 287.9    , 287.69598,
         287.086  ],
        ...,
        [278.248  , 278.25   , 278.348  , ..., 285.324  , 285.274  ,
         285.3    ],
        [278.148  , 278.148  , 278.176  , ..., 285.374  , 285.348  ,
         285.348  ],
        [278.074  , 278.122  , 278.05   , ..., 285.374  , 285.374  ,
         285.348  ]],

       [[286.058  , 286.18802, 285.426  , ..., 287.26   , 288.46002,
         288.284  ],
        [285.46002, 285.936  , 286.13   , ..., 287.858  , 287.812  ,
         287.488  ],
        [285.56198, 285.258  , 284.724  , ..., 287.86002, 287.68402,
         287.198  ],
...
        [279.8025 , 279.83667, 279.86334, ..., 279.     , 279.0375 ,
         279.03918],
        [279.6

# Tästä eteenpäin tehdään visualisointeja

In [136]:
# # Otetaan data ulos xarrayna, koska vain yksi käyttäjä DASK-järjestelmässä ja serverin käyttö hankaloittaa tekemistä porukalla
lat = EC_params['lat'].compute()
lon = EC_params['lon'].compute()
time = EC_params['time'].compute()

In [137]:
# Alla oleva tapa on "vanha" eli plotataan data lokaalilla alustalla.

In [138]:
def draw(parameter, t): 
    # Määritellään kartta:
    ax = generate_fig_properties(lat, lon)

    # Käsitellään dataa
    chart = meanT.isel(time=t)
    chart = chart - 273.15
    cmap = 'nipy_spectral'
    vmin = -40
    vmax = 40
    
    pcolormesh_map_plt(chart, vmin, vmax, cmap, lon, lat)

In [139]:
params = ['Floating 5 hour average temperature']
widgets.interact(draw, parameter=params, t=(0, len(time) - 1))

interactive(children=(Dropdown(description='parameter', options=('Floating 5 hour average temperature',), valu…

<function __main__.draw(parameter, t)>